# **Лабораторна робота №1. Системи DS**
# Система прогнозування цін на невживані автомобілі за їх характеристиками
# КМ-81. Юр'єва Ксенія
# Розуміння бізнесу — розуміння даних. Підготовка даних для вирішення конкретної задачі.

# **1.1 Розуміння бізнесу - розуміння даних**

Визначимося з предметною областю. Будемо досліждувати невживані автомобілі, їх види і показники (тип пального, розмір машини, тип двигуна тощо). Метою дослідження є підготовка та обробка даних для побудови моделей машинного навчання для прогнозування цін автомобілів. Основною задачею є прогнозування ціни автомобілю в залежності від його характеристик (тип пального, розмір машини, тип двигуна тощо). В результаті цієї лабораторної роботи, буде отримано підготований до створення і навчання моделей датасет. Результати дослідження та моделі можуть стати одним з способів оцінки цінової динаміки на ринку при випуску нових автомобілів.

# Збір данних з інтернет-ресурсів

Дані для обробки та подальшого використання в навчанні були взяті на сайті kaggle: https://www.kaggle.com/hellbuoy/car-price-prediction.

Імпорт модулів:

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression

 Шлях до данних має вигляд:

In [ ]:
path_to_car_data = '/kaggle/input/car-price-prediction/CarPrice_Assignment.csv'

# Збереження та опис даних

Збережемо дані з .csv в dataframe з бібліотеки pandas та виведемо їх:

In [ ]:
df_cars = pd.read_csv(path_to_car_data)

In [ ]:
df_cars

Виведемо типи даних, які належать колонкам таблиці.

In [ ]:
df_cars.info()

Розмір датасету:

In [ ]:
df_cars.shape

Опишемо основні з характеристики автомобілів, які наведені в датасеті:

* CarName (object) — повна назва автомобілю.
* fueltype (object) — тип пального.
* enginelocation (object) — розташування двигуна.
* enginetype (int64) — тип двигуна.
* curbweight (int64) — вага автомобіля.
* price (float64) — ціна автомобілю в доларах. Змінна, яку ми намагаєтеся передбачити за допомогою моделей.
і тд.

# 1.2 Підготування даних

# Первинна очистка даних, розвідувальний аналіз

На цьому етапі, проведемо підготовку иа очистку данних. На перший погляд, даний датасет немає пошкоджених даних, тому спочатку перевіримо цілісність данних і при необхідності очистимо їх. Потім, проаналізуємо, які з характеристик автомобілів впливають на ціну.

Приберемо назву автомобілю, замість цього у нас є Id (щоб не проводити зайвий LabelEncoding та працювати лише з числовими даними):

In [ ]:
df_cars.drop('CarName', axis = 1, inplace = True)

In [ ]:
df_cars

Оцінимо цілісність даних (виведемо кількість відсутніх рядків у кожному стовпчику):

In [ ]:
missing_data_ratio = pd.DataFrame({'Кількість втрачених даних': df_cars.isnull().sum()})
missing_data_ratio

Датасет немає проблем з цілісністю даних. Побудуємо кореляційну матрицю:

In [ ]:
corr_matrix = df_cars.corr()
plt.subplots(figsize=(12, 9))
sns.heatmap(corr_matrix, vmin = -1, vmax = 1, cmap="YlGnBu", annot = True, square=True)

Приберемо колонки, кореляція з ціною яких менша по модулю за 10%, адже вони не принесуть суттєвого впливу на результат моделі (скоріш за все, вони відносяться до зовнішнього виду авто, аніж до показників):

In [ ]:
not_corr_colls = ('symboling', 'stroke', 'compressionratio', 'peakrpm')
for col in not_corr_colls:
    df_cars.drop(col, axis = 1, inplace = True)

Дані з самого початку були очищенні та підготовані до обробки, а значить, можемо перейти до оціники основних статистичних характеристик (середнє, дисперсія і тд):

In [ ]:
df_cars.describe()

# Кодування ознак

На цьому етапі, закодуємо нечислові дані в числові за допомогою LabelEncoder() для спрощення роботи з даними в майбутньому та приберемо з датасету прогнозоване значення (price) та винесемо його в окремий датасет.

In [ ]:
str_cols = ('fueltype', 'aspiration', 'doornumber', 'carbody', 'drivewheel', 'enginelocation', 'fuelsystem',
            'highwaympg', 'enginetype', 'cylindernumber')
for col in str_cols:
    l = LabelEncoder()  # str -> float
    col_v = list(df_cars[col].values)
    df_cars[col] = l.fit(col_v).transform(col_v)

Окремо виведемо прогнозований показник, а саме ціну, та видалимо її з датасету

In [ ]:
y_ = df_cars.price.values
df_cars.drop('price', axis = 1, inplace = True)

# Розбиття набору даних

Розділимо дані на тренувальний та тестовий датасет (70% / 30%). Дане відношення (70% / 30%) на практиці є одним з найефективніших для навчання. Також, Для оцінки середньоквадратичного значення prediction моделі, нормалізуємо target значенння.

In [ ]:
split_percent = 0.8
df_cars_length = df_cars.shape[0]
y_shape = int(df_cars_length * split_percent)

X_train = df_cars[:y_shape]
X_test = df_cars[y_shape:]
y_train = y_[:y_shape]
y_test = y_[y_shape:]

y_train /= np.max(y_train)
y_test /= np.max(y_test)

# Побудова моделі (додатково)

Для перевірки даних, побудуємо на просту лінійну модель та виведемо показник для оцінки її точності під назвою RMSE.

In [ ]:
lin_model = LinearRegression()
lin_model.fit(X_train, y_train)

In [ ]:
preds = lin_model.predict(X_test)
RMSE = np.sqrt(np.sum((y_test - preds) ** 2))
print(RMSE)

Для прикладу, покажемо залежність ціни від кінських сил автомобіля: 

In [ ]:
df_cars['price'] = y_
sns.jointplot(data=df_cars, x="horsepower", y="price", kind="reg")
plt.show()

# Бізнес-профіль Еріксона-Пенкера

In [ ]:
from IPython.display import Image
Image(filename='/kaggle/input/penker/eriksen_penker (2).png') 

# 1.3 Верифікація та валідація результатів

# Верифікація

В даній лабораторній роботі була проведена підготовка даних для вирішення конкретної задачі, прогнозування цін невживаних авто. Необхідно було поставити задачу, визначивши предметну область, та дослідити дані, що були взяті з сайту kaggle.com.


* Предметна область - нові невживані авто.
* Задача - підготовка даних до побудови системи прогнозування ціни автомобілю в залежності від його характеристик (вага, тип двигуна, тип кузова, тип пального тощо).

Дані було успішно завантажено та збережено, також, була описана структура таблиці, її розмірність та обчислені основні статистичні характеристики даних в таблиці.

В процесі написання роботи та обробки даних, які були цілісні, було проведено:
* Кореляційний аналіз
* Усунення несуттєвих або некорельованих значень
* Розраховано основні статистичні характеристики
* Кодування ознак
* Dataframe розбито на тренувальну та тестову вибірку

Також, задокументовано класи бізнес-профілю Еріксона-Пенкера, а отже, верифікація пройшла успішно.

# Валідація

Обробивши датасет, можна сказати, що дані є цілісними. Проведено аналіз ознак та впливу їх на ціну автомобіля. 
В даному датасеті майже всі дані мали достатній вплив на ціну. Також, було виявлено, які показники більше всьго впливають на ціну автомобілю (розмір двигуна та кінські сили). Дивлячись на високий рівень кореляції кожного з показників можна сказати, що вони підходять для точного вирішення задачі прогнозування.

# 1.4 Висновки

В результаті лабораторної роботи, було отримано підготовлені до тренування дані та прості моделі для прогнозування значень ціни нового автомобіля, які побудовані на підготовлених в даній лабораторній даних.

Обробка данних за допомогою математичного апарату показала нам, що найбільше ціна залежить від розміру двигуна, маси, кылькосты обертів за хвилину, кінських сил. Наприклад, чим більший розмір двигуна, потужність двигуна (кінські сили), тим більше автомобіль коштуватиме. Найменше впливають на вартість висота авто, хід поршня та ступінь стиску двигуна внутрішнього згоряння.

В процесі підготовки даних ми скористалися:
* Python
* Бібліотеки для статистики та роботи з данними (numpy, pandas, sklearn, matplotlib, seaborn).


Було проведено основні етапи створення систем DS:
* Збір даних з ресурсу
* Обробка одержаних даних
* Видалення не потрібних для аналізу даних
* Розвідувальний та кореляційний аналіз важливих даних
* Кодування ознак
* Розбиття на тренувальні та тестові набори для виконання наступної лабораторної роботи
* Верифікація та валідація результатів

В результаті, дані були підготовані до наступного етапу розробки - прогнозування цін нових невживаних авто. Для перевірки, було побудовано просту лінійну модель. Верифікація і валідація пройшли успішно, під час перевірки проблем не було виявлено, отже, дані готові до використання на наступних етапах роботи.